# Load data

In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
df = pd.read_csv('../data/schedules_1.csv', index_col=0).astype(int).T
df.head()

,p,w,r
1,2,2,1
2,3,8,2
3,2,10,3


# Definition of functions

In [3]:
def cover_all_zeros(lap, modify_input=False):
    if len(lap) == 0:
        return (0, [], [])
    
    if not modify_input:
        lap = np.copy(lap)
    
    zero_matrix = (lap == 0).astype(int)
    
    row_counter = np.array([sum(row) for row in zero_matrix])
    col_counter = np.array([sum(row) for row in zero_matrix.transpose()])
    
    row_i = np.where(row_counter == np.amax(row_counter))[0][0]
    col_i = np.where(col_counter == np.amax(col_counter))[0][0]
    
    max_row_zeros = row_counter[row_i]
    max_col_zeros = col_counter[col_i]
    
    if max_row_zeros == 0 and max_col_zeros == 0:
        return (0, [], [])
    if max_row_zeros >= max_col_zeros:
        lap = np.delete(lap, row_i, axis=0)
        
        nzeros, zrows, zcols = cover_all_zeros(lap, True)
        nzeros += 1
        
        zrows = [(x if x < row_i else x + 1) for x in zrows]
        zrows.append(row_i)
    else:
        lap = np.delete(lap, col_i, axis=1)
        
        nzeros, zrows, zcols = cover_all_zeros(lap, True)
        nzeros += 1
        
        zcols = [(x if x < col_i else x + 1) for x in zcols]
        zcols.append(col_i)
    
    return (nzeros, zrows, zcols)

In [4]:
def create_additional_zeros(lap, uncovered_rows):
    # Find the smallest element (call it k) that is not covered by a line
    smallest_row_i = -1
    smallest_row_min = -1
    for i in uncovered_rows:
        row = lap[i]
        row_indices = np.where((row != 0) & (row != np.inf))
        
        current_smallest_row_min = min(np.take(lap[i], row_indices)[0])
        if smallest_row_min == -1 or smallest_row_min > current_smallest_row_min:
            smallest_row_i = i
            smallest_row_min = current_smallest_row_min
    
    # Substract smallest element
    for row_i in uncovered_rows:
        lap[row_i] = lap[row_i] - smallest_row_min
    
    # Locate cols to fix
    fix_col_indices = np.where(lap[smallest_row_i] < 0)[0]
    for row_i, row in enumerate(lap):
        lap[row_i] = [(x + smallest_row_min if col_i in fix_col_indices else x) for col_i, x in enumerate(row)]
    
    return lap

In [5]:
def hungarian_algorithm(lap):
    size = len(lap)
    hlap = np.copy(lap)
    
    # Substract row minimum
    for i, row in enumerate(hlap):
        hlap[i] = row - min(row)
    
    # Substract column minimum
    hlap = hlap.transpose()
    for i, row in enumerate(hlap):
        hlap[i] = row - min(row)
    hlap = hlap.transpose()
    
    # Cover all zeros with a minimum number of lines
    nzeros, zrows, zcols = cover_all_zeros(hlap)
    while nzeros < size:
        # Find the smallest element (call it k) that is not covered by a line in Step 3
        # Subtract k from all uncovered elements, and add k to all elements that are covered twice
        all_indices = np.arange(size)
        uncovered_rows = np.delete(all_indices, zrows, axis=0)
        hlap = create_additional_zeros(hlap, uncovered_rows)
        
        # Cover all zeros with a minimum number of lines
        nzeros, zrows, zcols = cover_all_zeros(hlap)
    
    # To do: return combination
    return hlap

In [6]:
# Finds an assignation where each part job is assigned to a time
# It does not verify if the given result is feasible or not
def find_permutation(lap, permutation=[]):
    if len(permutation) == len(lap):
        return permutation
    
    row_i = len(permutation)
    row = lap[row_i]
    
    # Find all columns (in the current row) where there are zeros
    zero_indices = np.where(row == 0)[0]

    for zero_i in zero_indices:
        # Skip column if it is already chosen
        if zero_i in permutation:
            continue

        copy_permutation = permutation.copy()
        copy_permutation.append(zero_i)

        copy_permutation = find_permutation(lap, copy_permutation)

        # Find a permutation and return it
        if len(copy_permutation) != 0:
            return copy_permutation
        
    return []

In [7]:
# Given a permutation, calculate the cost
def get_cost(lap, permutation):
    cost = 0
    for row_i, row in enumerate(lap):
        col_i = permutation[row_i]
        cost += row[col_i]
    
    return cost

In [8]:
# In a permutation, it cannot happen that a later job part is executed before a earlier job part
# Here it returns the choices in LAP which are not feasible
def find_infeasible_choice(job_parts, permutation):
    last_job_i = -1
    last_choice = -1
    
    for row_i in range(len(permutation)):
        current_choice = permutation[row_i]
        current_job_i = job_parts[row_i]
        
        if last_job_i == current_job_i:
            if last_choice > current_choice:
                return [
                    (row_i - 1, last_choice),
                    (row_i, current_choice)
                ]
        else:
            last_job_i = job_parts[row_i]
        last_choice = current_choice
    return None

In [21]:
# Tree path is defined by the coordinates in the LAP which has been excluded
def find_permutation_min_cost(lap, hlap=None, prev_cost=-1):
    if hlap is None:
        # Execute only in the first call
        hlap = lap
    
    # Compute Hungarian algorithm and find a permutation of jobs
    hlap = hungarian_algorithm(hlap)
    permutation = find_permutation(hlap)
    
    if len(permutation) == 0:
        # No permutation found
        return (None, -1)
    
    cost = get_cost(lap, permutation)
    if cost >= prev_cost and prev_cost != -1:
        # Too expensive
        return (None, -1)
    
    infeasible_choice = find_infeasible_choice(job_parts, permutation)
    
    if infeasible_choice is None:
        # This is a feasible permutation
        return (permutation, cost)
    
    # If there is any infeasible choice in the permutation, re-compute it again
    best_permutation = None
    best_cost = -1
    for choice in infeasible_choice:
        # Replace is each infeasible choice with np.inf and compute again
        copy_hlap = copy.deepcopy(hlap)
        copy_hlap[choice[0]][choice[1]] = np.inf
        
        current_permutation, current_cost = find_permutation_min_cost(lap, copy_hlap, best_cost)
        
        if current_permutation is None:
            continue
        elif current_cost < best_cost or best_cost == -1:
            best_permutation = current_permutation
            best_cost = current_cost
    
    return (best_permutation, best_cost)

# Reduction of SPT to the Linear Assignment Problem

In [22]:
T = df['p'].sum()
T

7

In [23]:
lap = []
job_parts = []

job_i = 0
for _, job_data in df.iterrows():
    for p_i in range(job_data['p']):
        # Add job name to each row, so we know which row is a part of which job
        job_parts.append(job_i)
        
        # Compute LAP
        lap_row = np.ones(T) * np.inf
        
        start = job_data['r'] + p_i - 1
        end = T - job_data['p'] + p_i + 1
        mult = 1
        for index in range(start, end):
            lap_row[index] = int(job_data['w'] * mult) if p_i + 1 == job_data['p'] else 0
            mult += 1
        
        lap.append(lap_row)

    job_i += 1


lap = np.array(lap)
lap

array([[ 0.,  0.,  0.,  0.,  0.,  0., inf],
       [inf,  2.,  4.,  6.,  8., 10., 12.],
       [inf,  0.,  0.,  0.,  0., inf, inf],
       [inf, inf,  0.,  0.,  0.,  0., inf],
       [inf, inf, inf,  8., 16., 24., 32.],
       [inf, inf,  0.,  0.,  0.,  0., inf],
       [inf, inf, inf, 10., 20., 30., 40.]])

In [24]:
best_permutation, best_cost = find_permutation_min_cost(lap)

In [25]:
print('Best permutation: ', best_permutation)
print('Cost: %.1f' % best_cost)

Best permutation:  [0, 1, 4, 5, 6, 2, 3]
Cost: 44.0
